In [ ]:
pip install fastparquet


In [ ]:
import pandas as pd

# Cargar el archivo Parquet en un DataFrame
df = pd.read_parquet('../Datasets/user_reviews.parquet')

In [ ]:
df


In [ ]:
#Limpiar todas las variables definidas
for variable in list(locals()):
    del locals()[variable]

In [ ]:
# Guarda el DataFrame df_reviews en formato CSV en el archivo 'user_reviews.csv' dentro de la carpeta 'Datasets', excluyendo la escritura de índices
df_reviews.to_csv('Datasets/user_reviews.csv', index=False)

In [ ]:
import pickle

# Guardar el objeto en un archivo
with open('df_items.pickle', 'wb') as file:
    pickle.dump(df_items, file)

In [ ]:
import pickle

# Recuperar el objeto desde el archivo
with open('df_items.pickle', 'rb') as file:
    df_items = pickle.load(file)

In [ ]:
pip install TextBlob

In [ ]:
def PlayTimeGenre(genero):
    # Filtrar el DataFrame de juegos por el género específico
    juegos_genero = df_games[df_games['genres'].str.contains(genero, case=False)]

    if juegos_genero.empty:
        return json.dumps({f"No hay datos para el género {genero}": None}, ensure_ascii=False)

    # Filtrar el DataFrame de items por los item_id de juegos_genero
    items_genero = df_items[df_items['item_id'].isin(juegos_genero['item_id'])]

    if items_genero.empty:
        return json.dumps({f"No hay datos de juego para el género {genero}": None}, ensure_ascii=False)

    # Agrupar y sumar las horas jugadas por año sin necesidad de fusión
    grouped = items_genero.merge(juegos_genero, on='item_id').groupby('release_year')['playtime_forever'].sum()

    if grouped.empty:
        return json.dumps({f"No hay datos de horas jugadas para el género {genero}": None}, ensure_ascii=False)

    # Encontrar el año con más horas jugadas
    anio_mas_jugado = grouped.idxmax()

    # Formatear el resultado como un diccionario y convertirlo a JSON
    resultado = {f"Año de lanzamiento con más horas jugadas para el género {genero}": str(anio_mas_jugado)}
    return json.dumps(resultado, ensure_ascii=False)

In [ ]:
def UserForGenre1(genero):
    # Filtrar columnas no necesarias desde el principio
    relevant_df_games = df_games[['item_id', 'genres', 'release_year']]

    # Filtrar datos solo para el género deseado
    genre_filter = relevant_df_games[relevant_df_games['genres'].str.contains(genero, case=False)]

    # Verificar si hay datos para el género dado
    if genre_filter.empty:
        return json.dumps({"message": f"No hay datos para el género '{genero}'"}, ensure_ascii=False)

    # Merge con df_items solo con columnas relevantes
    merged_df = pd.merge(genre_filter, df_items[['item_id', 'playtime_forever', 'user_id']], on='item_id')

    # Encontrar el usuario con más horas jugadas
    if 'user_id' in merged_df.columns:
        user_max_hours = merged_df.groupby('user_id')['playtime_forever'].sum().idxmax()

        # Filtrar para el usuario con más horas jugadas
        user_info = merged_df[merged_df['user_id'] == user_max_hours]

        # Calcular la acumulación de horas jugadas por año
        hours_per_year = user_info.groupby('release_year')['playtime_forever'].sum().reset_index()
        hours_per_year['playtime_forever'] = (hours_per_year['playtime_forever'] / 60).astype(int)  # Convertir minutos a horas

        # Filtrar las filas con valores cero
        hours_per_year = hours_per_year[hours_per_year['playtime_forever'] != 0]

        # Renombrar columnas
        hours_per_year = hours_per_year.rename(columns={'release_year': 'Año:', 'playtime_forever': 'Horas:'})

        # Estructurar los resultados en un diccionario
        result = {
            f"Usuario con más horas jugadas para Género {genero}": user_max_hours,
            "Horas jugadas": hours_per_year.to_dict(orient='records')
        }

        # Convertir el diccionario a formato JSON
        json_result = json.dumps(result, ensure_ascii=False, indent=4)
        return json_result
    else:
        return json.dumps({"message": "No se encontró información de horas jugadas para el género dado"}, ensure_ascii=False)

In [ ]:
# Llamada a la función con el género deseado, por ejemplo 'Shooter'
resultado_json = UserForGenre1('indie')
print(resultado_json)  # Imprimir el resultado JSON